# DBSCAN

For this practical we will be considering a dataset containing the credit card usage behavior of about 9000 active credit card holders during a 6 month period. 

We will use DBSCAN to cluster the activity and see if it corresponds to groups of spending types. 


### The data dictionary

The data contains 18 variables being the following:

| Variable                       | Description                                                                                                                 |
|--------------------------------|-----------------------------------------------------------------------------------------------------------------------------|
| CUSTID                         | Identification of Credit Card holder (Categorical)                                                                          |
| BALANCE                        | Balance amount left in their account to make purchases                                                                      |
| BALANCEFREQUENCY               | How frequently the balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated)           |
| PURCHASES                      | Amount of purchases made from account                                                                                       |
| ONEOFFPURCHASES                | Maximum purchase amount done in one-go                                                                                      |
| INSTALLMENTSPURCHASES          | Amount of purchase done in installment                                                                                      |
| CASHADVANCE                    | Cash in advance given by the user                                                                                           |
| PURCHASESFREQUENCY             | How frequently the purchases are being made, score between 0 and 1 (1 = frequently purchased, 0 = not frequently purchased) |
| ONEOFFPURCHASESFREQUENCY       | How frequently purchases are happening in one-go (1 = frequently purchased, 0 = not frequently purchased)                   |
| PURCHASESINSTALLMENTSFREQUENCY | How frequently purchases in installments are being done (1 = frequently done, 0 = not frequently done)                      |
| CASHADVANCEFREQUENCY           | How frequently the cash in advance is being paid                                                                            |
| CASHADVANCETRX                 | Number of transactions made with "Cash in Advance"                                                                          |
| PURCHASESTRX                   | Number of purchase transcations made                                                                                        |
| CREDITLIMIT                    | Limit of credit card for user                                                                                               |
| PAYMENTS                       | Amount of payments made by the user                                                                                         |
| MINIMUM_PAYMENTS               | Minumum amount of the payments made by the user                                                                             |
| PRCFULLPAYMENT                 | Percentage of full payments which has been paid by the user                                                                 |
| TENURE                         | Tenure of credit card service for the user                                                                                  |

Source is [here](https://www.kaggle.com/arjunbhasin2013/ccdata).

In [ ]:
# Import the necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances

# Turn off pink warnings

import warnings
warnings.filterwarnings('ignore')

## 1) Load and transform the data


Read the data contained in `data/CC GENERAL.csv` into a pandas dataframe called `df`, whilst also dropping the column called `CUST_ID`. Then take a look at the data (via the `.head`) method to check the formatting.

Check if there are null values in the data using the function `isnull()`.

We find that there are some missing values but it is a very small proportion of the dataset and only present in 2 columns. Therefore, drop the rows with missing values and store this in `df`.

In [ ]:
# Handling the missing values 


We can now check the data type for each column and confirm that it is as expected.

In [ ]:
# Check the data types


## 2) Explore the data

Using the `describe` function, explore the statistical properties of the dataset. By considering the 75th percentile and the maximum values, what can we say about certain columns of the data?

There are large outliers in the columns: `BALANCE`, `PURCHASES`, `ONEOFF_PURCHASES`, `INSTALLMENTS_PURCHASES`, `CASH_ADVANCE`, `CREDIT_LIMIT`, `PAYMENTS` and `MINIMUM_PAYMENTS`. We can see this by approximately considering if the maximum or minimum values are more than the mean plus/minus twice the standard deviation.

Therefore we will fit the DBSCAN model to the full dataset and scale our data to handle the skew. As an extension, we later consider a simple approach to handle this by dividing the data into categories. 

Extension: [This](https://towardsdatascience.com/a-brief-overview-of-outlier-detection-techniques-1e0b2c19e561) is a good article on common outlier detection techniques. The z-score provides an easy way to quickly identify possible outliers, however note that it does assume a normal distribution for your data.

## 3) Scaling the data for clustering

Remember that when considering the distance between points it is important to scale the data. This is so that we don't increase the importance of variables which have larger values by nature. 

For this dataset we will rescale the data using the following steps:

1) Using `StandardScaler`, rescale the data and save it in a variable called `scaled_data`.
    Hint: For this define a `StandardScaler` object and use it with the `fit_transform` function on our dataframe. Store the scaled data as `scaled_data`.
  
2) Convert `scaled_data` to a pandas dataframe with the same columns as `df`. Store this as `scaled_df`.

In [ ]:
# 1) Scale the data to bring all the attributes to a comparable level 

# 2) Converting to a pandas DataFrame 


Let us quickly check that the `StandardScaler` has worked by using `describe` to check the new means and standard deviations. Our scaled data should now have a mean of zero (or as close as a machine can get!) and a standard deviation of 1. 

In [ ]:
scaled_df.describe()

## 4) Investigate a starting value for eps

Measure the distance of each point to its closest neighbor. To do this, use the function `sklearn.metrics.pairwise.pairwise_distances` on the scaled dataset, `scaled_df`. Let the output of `pairwise_distances` be stored as `all_distances`, and store the list of neighbour distances as `neig_distances`.

Note that the documentation is given [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html).

Plot the distribution of the distances using the function `hist`. Use enough bins to clearly see the behaviour at the tail, around 100-200 should be sufficient. 

In [ ]:
# Plot the distances


Looking at your graph what distances look appropriate for eps? Remember we want to choose a value slightly higher than the optimal one and then revise it down, so we'll choose an intial value to the right of the distribution. 

## 5) Principal Component Analysis (PCA) of data

Initially we will just use these two principal components to easily plot the data, however later in the exercise we will also explore how we can apply DBSCAN to them directly. 

Get the first 2 principal components of the data. To do this, use the function `PCA` to create a `pca` object and then use the `fit_transform` function to apply it to the scaled data. Store the result in a dataframe called `X_principal` with columns `P1` and `P2`. 

## 6) Fit the DBSCAN model 

Apply DBSCAN, starting with `eps=2` and `min_samples=34` using the functions `DBSCAN` and `fit_predict`, storing the output as `cluster_assignment`. Then print out the unique clusters found. Note that the cluster labelled as `-1` are identified as noise/outliers by the DBSCAN model.

Remember to apply the function to `scaled_df`. You can be explicit in specifying the columns of interest by using `.iloc[0:17]`. This is so that if adding columns containing the cluster assignments to our dataframe, we don't include them in any modelling later on.

Note: We start with `eps=2` as this sits toward the right tail of our histogram of distances so this is a suitable first choice. We start with `min_samples=34` as a good rule of thumb is to start with twice the number of features, here ` 2 x 17`, this can then be increased if needed. 

Assign a column called `labels` to `scaled_df` containing the cluster assignment

Create a scatter plot with `x=X_principal['P1']` and `y=X_principal['P2']` and set the colour parameter `c=cluster_assignment`. Also change the size of the points using `s=3`, this will make it clearer to see the dense regions.

We can see that our model has captured one cluster and termed the remaining points as noise. These two groups seem to overlap when we visualise them using the first two principle components, why do you think this is?

We can now explore the distribution of each variable for each of the clusters identified. Use the below code to explore these properties and see what conclusions we can make about each cluster. 

In [ ]:
for c in scaled_df.iloc[:,0:17]:
    grid= sns.FacetGrid(scaled_df, col='labels')
    grid.map(plt.hist, c)


It looks like the non-noise cluster corresponds to a specific type of customer behaviour. This could be used in several key ways including seeing if an anomalous payment was made by a customer based on their behaviour cluster. Alternatively on a business level, different marketing strategies can be pitched to each group in order to maximise results. 

Specifically for this clustering outcome we find that: 

Cluster 0: Customers with a typically low one-off purchase frequency and consistently high tenure. 

Note: this clustering was produced using our initial guesses of `eps=2` and `min_samples=34`. As we've done nothing to tune these values they're likely not optimal and probably don't accurately represent the underlying patterns or groups within the data.

## 7) Additional modelling

We can also consider running DBSCAN on the PCA components. To do this run the same DBSCAN as previously but on the dataset `X_principal` using the same values for `eps` and `min_samples` and store the output once again in `cluster_assignment`.

Then print out the unique clusters

Plot the results with the new cluster assignment. As before, create a scatter plot of the principal components, coloured by the new cluster assignment.

Then plot the distribution of the results to identify the interpretation of each cluster in terms of the credit card user's behaviour. To do this: 

1) Assign the new cluster assignment to a column of `scaled_df` called `pca_labels`. 

2) Then copy the code used previously for the distributional plots, changing the `col` parameter to be the new `pca_labels`.

This time nearly all of the points have been assigned to the cluster with very few points being classed as outliers. Even though our model hyperparameters have remained the same our output has changed, this is because the modelling was perfomed on the 2 dimensional PCA data space which is denser. Why do you think the PCA space is denser than the original space? How might we change our `eps` and `min_values` given that we've moved to a denser space?

## Extension: Handling skewed data

As noticed previously, there are large outliers in the columns: `BALANCE`, `PURCHASES`, `ONEOFF_PURCHASES`, `INSTALLMENTS_PURCHASES`, `CASH_ADVANCE`, `CREDIT_LIMIT`, `PAYMENTS` and `MINIMUM_PAYMENTS`.

We are interested in finding similarities through clusters so it is helpful to group values in these columns with large outliers to categories. We'll then replace the category labels with the median value of each category, this allows us to preserve some information while (hopefully!) removing skew. 

Visualise this by first plotting the distribution of `df['PURCHASES']` using the `hist` function.

Print the maximum value in the column `Purchases`.

As we can see, there is a very large spread of values. Therefore first create a copy of `df` and name it `categorical_df`.

Run the below code to convert the columns listed to categories. The new columns will be stored in columns with the same name, but with `_MEDIAN` appended. 

In [ ]:
columns=['BALANCE', 'PURCHASES', 'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE', 'CREDIT_LIMIT',
                'PAYMENTS', 'MINIMUM_PAYMENTS']

categorical_df = df.copy()

for c in columns:

    Range=c+'_MEDIAN'
    categorical_df[Range]=0        
    categorical_df.loc[((categorical_df[c]>0)&(categorical_df[c]<=500)),Range]=1
    categorical_df.loc[((categorical_df[c]>500)&(categorical_df[c]<=1000)),Range]=2
    categorical_df.loc[((categorical_df[c]>1000)&(categorical_df[c]<=3000)),Range]=3
    categorical_df.loc[((categorical_df[c]>3000)&(categorical_df[c]<=5000)),Range]=4
    categorical_df.loc[((categorical_df[c]>5000)&(categorical_df[c]<=10000)),Range]=5
    categorical_df.loc[((categorical_df[c]>10000)),Range]=6

Run the below code to replace each category with the median value from the original column.

Note: `c[:-7]]` removes the "_MEDIAN" from our column names allowing us to access the original column.

In [ ]:
for c in categorical_df.columns[-len(columns):]:
    for i in range(1, 7):
        col_median = categorical_df.loc[categorical_df[c]==i, c[:-7]].median()
        categorical_df.loc[categorical_df[c]==i, c] = col_median

categorical_df.head()

We can check how the summary statistics have changed by using `describe` on the transformed data and original data.

In [ ]:
categorical_df.iloc[:,-len(columns):].describe()

In [ ]:
categorical_df[columns].describe()

As expected the max values for each column have been reduced while the means have remained mostly the same. 

Then drop the columns given by the list `columns` from the dataframe `categorical_df`.

Check that we have the same number of features in our transformed dataframe as in our original by comparing the `shape`.

As previously, we can now scale the data contained in `categorical_df`. Store the result in `scaled_cat_df`.

In [ ]:
# 1) Scale the data to bring all the attributes to a comparable level 

# 2) Converting to a pandas DataFrame 


Compute the first two principal components of the data contained in `scaled_cat_df` and save this in a dataframe called `X_cat_principal`. You can see the first rows using the `head` function. We'll use this to plot our new clusters.

Find a starting value for eps by using the distribution of pairwise distances of the principal component dataframe `X_principal` and plot the results as a histogram. 

Starting with `eps=1.1` and `min_samples=34`, run the DBSCAN algorithm on `scaled_cat_df` and print out the unique numbers of clusters.

Run `value_counts` on a Series containing the `cluster_assignment`. Does this look like a more promising clustering attempt?

Plot the result in a scatter plot of the principal components and coloured by the clustering assignment.

Finally use the below code to explore the results and interpretation of each cluster. You can toggle the `eps` and `min_samples` parameters to see how the clusters can vary both visually and distributionally. 

In [ ]:
scaled_cat_df['pca_cat_labels'] = cluster_assignment
for c in scaled_cat_df.iloc[:,0:17]:
    grid= sns.FacetGrid(scaled_cat_df, col='pca_cat_labels')
    grid.map(plt.hist, c)

This is just one way of dealing with skewed data when performing cluster analysis. For example another approach would be to take a log transform of our data before scaling it. 